# Setup Verification Notebook

This notebook verifies that your environment is properly configured for the LLM Evaluations Workshop.

## Prerequisites
1. Python virtual environment activated
2. Required packages installed (`pip install -r requirements.txt`)
3. Azure OpenAI credentials configured in `.env` file
4. Azure AI Foundry project set up

## 1. Import Dependencies

In [ ]:
# Test core package imports
try:
    import azure.ai.evaluation
    from openai import AzureOpenAI
    from azure.identity import DefaultAzureCredential
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import json
    import os
    from dotenv import load_dotenv
    
    print("✅ All required packages imported successfully!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please run: pip install -r requirements.txt")

## 2. Environment Configuration

In [ ]:
# Load and verify environment variables
load_dotenv()

required_env_vars = [
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_KEY',
    'AZURE_OPENAI_DEPLOYMENT_NAME'
]

print("Environment Variables Check:")
for var in required_env_vars:
    value = os.getenv(var)
    if value:
        # Mask the API key for security
        if 'KEY' in var:
            display_value = f"{value[:8]}...{value[-4:]}" if len(value) > 12 else "[MASKED]"
        else:
            display_value = value
        print(f"✅ {var}: {display_value}")
    else:
        print(f"❌ {var}: Not set")
        print(f"   Please add {var} to your .env file")

## 3. Azure OpenAI Connection Test

In [ ]:
# Test Azure OpenAI connection
try:
    from shared_utils.azure_clients import azure_manager
    
    # Get the client
    client = azure_manager.get_openai_client()
    
    # Test with a simple completion
    response = client.chat.completions.create(
        model=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'),
        messages=[
            {"role": "user", "content": "Say 'Hello, Workshop!' if you can hear me."}
        ],
        max_tokens=50
    )
    
    print("✅ Azure OpenAI connection successful!")
    print(f"Response: {response.choices[0].message.content}")
    print(f"Model: {response.model}")
    print(f"Tokens used: {response.usage.total_tokens}")
    
except Exception as e:
    print(f"❌ Azure OpenAI connection failed: {e}")
    print("Please check your Azure OpenAI configuration in .env file")

## 4. Evaluation SDK Test

In [ ]:
# Test Azure AI Evaluation SDK
try:
    from azure.ai.evaluation import RelevanceEvaluator, evaluate
    
    # Get model configuration
    model_config = azure_manager.get_model_config()
    
    # Create a simple evaluator
    relevance_evaluator = RelevanceEvaluator(model_config=model_config)
    
    # Test data
    test_data = [{
        "query": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "context": "France is a country in Western Europe. Its capital and largest city is Paris."
    }]
    
    # Run evaluation
    result = evaluate(
        data=test_data,
        evaluators={"relevance": relevance_evaluator}
    )
    
    print("✅ Azure AI Evaluation SDK working!")
    print(f"Relevance score: {result['metrics']['relevance']}")
    
except Exception as e:
    print(f"❌ Evaluation SDK test failed: {e}")
    print("This might be expected if you don't have AI Foundry set up yet")

## 5. Shared Utilities Test

In [ ]:
# Test shared utilities
try:
    from shared_utils.data_utils import create_sample_dataset
    from shared_utils.evaluation_helpers import format_evaluation_summary
    
    # Create a sample dataset
    sample_data = create_sample_dataset(size=3)
    print("✅ Sample dataset created:")
    for i, item in enumerate(sample_data[:2]):  # Show first 2 items
        print(f"  {i+1}. Q: {item['question'][:50]}...")
        print(f"     A: {item['answer'][:50]}...")
    
    print(f"\n✅ Shared utilities working correctly!")
    
except Exception as e:
    print(f"❌ Shared utilities test failed: {e}")

## 6. Directory Structure Verification

In [ ]:
# Verify expected directory structure
import os
from pathlib import Path

workshop_root = Path('.')
expected_structure = [
    'requirements.txt',
    '.env.template',
    'shared_utils/',
    'shared_utils/azure_clients.py',
    'shared_utils/evaluation_helpers.py',
    'shared_utils/data_utils.py'
]

print("Directory Structure Check:")
for item in expected_structure:
    path = workshop_root / item
    if path.exists():
        print(f"✅ {item}")
    else:
        print(f"❌ {item} - Missing")

# Check for .env file
env_file = workshop_root / '.env'
if env_file.exists():
    print("✅ .env file exists")
else:
    print("⚠️  .env file not found - copy .env.template to .env and configure")

## Summary

If all checks above show ✅, you're ready to proceed with the workshop!

### Next Steps:
1. **Lab 1**: Start with `lab1_evaluation_fundamentals.ipynb`
2. **Documentation**: Check the `docs/` folder for additional resources
3. **Support**: Open an issue in the GitHub repository if you encounter problems

### Common Issues:
- **Import errors**: Run `pip install -r requirements.txt`
- **Azure connection issues**: Verify your `.env` configuration
- **Permission errors**: Ensure your Azure account has proper permissions

Happy learning! 🚀